<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Inventic_AI_Engineer_Assignment_Mohd_Kaif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.7 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display, clear_output
import json

In [3]:
# Load models for word tokenization, emotion analysis, book suggestions, and summarization
word_tokenizer = pipeline("token-classification", model="dslim/bert-base-NER")
emotion_classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion")
book_suggester = pipeline("text-generation", model="gpt2")  # Use GPT-2 for book suggestions
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
def count_words(passage):
    """
    Counts the total number of words in the passage using a tokenizer.

    Args:
        passage (str): The input text passage.

    Returns:
        int: The total number of words.
    """
    tokens = word_tokenizer(passage)
    return len([token for token in tokens if token['entity'].startswith('B-') or token['entity'].startswith('I-')])

In [5]:
def analyze_emotion(passage):
    """
    Analyzes the predominant emotion in the passage using a pre-trained model.

    Args:
        passage (str): The input text passage.

    Returns:
        str: The predominant emotion (e.g., joy, sadness, anger, neutral).
    """
    result = emotion_classifier(passage)[0]
    return result['label']

In [6]:
def suggest_books(passage):
    """
    Suggests 2-3 possible books using a text-generation model.

    Args:
        passage (str): The input text passage.

    Returns:
        list: A list of possible books.
    """
    prompt = f"Suggest 2-3 books that this passage might be from: '{passage}'"
    suggestions = book_suggester(prompt, max_length=100, num_return_sequences=1, do_sample=True)
    return suggestions[0]['generated_text'].strip().split("\n")[:3]  # Return top 3 suggestions

In [7]:
def summarize_passage(passage):
    """
    Summarizes the passage using a pre-trained model.

    Args:
        passage (str): The input text passage.

    Returns:
        str: A 2-3 sentence summary.
    """
    summary = summarizer(passage, max_length=50, min_length=25, do_sample=False)
    return summary[0]['summary_text']

In [8]:
def analyze_passage(passage):
    """
    Analyzes the passage and returns the results in a structured format.

    Args:
        passage (str): The input text passage.

    Returns:
        dict: A dictionary containing the analysis results.
    """
    results = {
        "word_count": count_words(passage),
        "predominant_emotion": analyze_emotion(passage),
        "possible_books": suggest_books(passage),
        "summary": summarize_passage(passage)
    }
    return results

In [9]:
# Text area for input passage
passage_input = widgets.Textarea(
    value='',
    placeholder='Enter the passage you\'d like to analyze...',
    description='Passage:',
    disabled=False,
    layout=widgets.Layout(width='80%', height='150px')
)

# Button to trigger analysis
analyze_button = widgets.Button(
    description='Analyze Passage',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to analyze the passage',
    icon='check'
)

# Output area for results
output = widgets.Output()

# Function to handle button click event
def on_analyze_button_click(b):
    with output:
        clear_output()  # Clear previous output
        passage = passage_input.value.strip()
        if passage:
            print("Analyzing passage... Please wait.")
            results = analyze_passage(passage)
            print("\n--- Analysis Results ---")
            print(json.dumps(results, indent=4))

            # Option to save results to a file
            save_results = input("Would you like to save the results to a file? (yes/no): ").strip().lower()
            if save_results == "yes":
                file_name = input("Enter the file name (e.g., results.json): ").strip()
                with open(file_name, 'w') as file:
                    json.dump(results, file, indent=4)
                print(f"Results saved to {file_name}")
        else:
            print("No valid input provided.")

# Attach the button click event
analyze_button.on_click(on_analyze_button_click)

# Display the input text area and button
display(passage_input, analyze_button, output)

Textarea(value='', description='Passage:', layout=Layout(height='150px', width='80%'), placeholder="Enter the …

Button(button_style='success', description='Analyze Passage', icon='check', style=ButtonStyle(), tooltip='Clic…

Output()